In [ ]:
import numpy as np

def get_scan_angle(ra, dec, epoch, dir_of_normal_vector):
    ## get the projection factor in RA and DEC direction, which is the same as tan(theta) and cot(theta) in proper definition of theta. 
    ## a_projection**2 + d_projection**2 = 1
    ## note that RA increase from right to left
    ## ra, dec, epoch are 1D np.array of the same length. All epochs of all objects should be stacked into 1D:
    ## e.g., given 2 objects with 4 and 3 epochs respectively, they should follow the form
    ## ra = np.array([a1,a1,a1,a1,a2,a2,a2]), dec = np.array([d1,d1,d1,d1,d2,d2,d2]), 
    ## epoch = np.array([t11,t12,t13,t14,t21,t22,t23]), where all symbols in the arrays are float
    ## epoch follows the definition of the 'time' column in light-curve catalogue, ranging from 1666 to 2336 [day]

    xyz = np.load(dir_of_normal_vector+'norm_xyz.npy') # x, y, z of the normal vector of the scan-plane
    position_xyz = np.concatenate([(np.cos(ra/180*np.pi) * np.cos(dec/180*np.pi)).reshape(-1,1),\
                (np.sin(ra/180*np.pi) * np.cos(dec/180*np.pi)).reshape(-1,1), \
                           np.sin(dec/180*np.pi).reshape(-1,1)],axis=1)  # position vector of the source


    interval_index = np.zeros(len(epoch),dtype=int)
    interval_index[:] = (np.round(epoch)-1666)
    xyz_each = np.empty((len(epoch),3))
    for i in range(len(epoch)):
        xyz_each[i,:] = xyz[interval_index[i],:]
    tangential = np.cross(xyz_each,position_xyz)  # a vector in the direction of scan (or maybe the anti-direction)
    tangential = tangential/np.sqrt((tangential**2).sum(axis=1).reshape(-1,1)) # normalize the vector
    alpha_axis = np.cross((0,0,1),position_xyz)
    delta_axis = np.cross(position_xyz,alpha_axis)
    alpha_axis = alpha_axis/np.sqrt((alpha_axis**2).sum(axis=1).reshape(-1,1))  # normalize the vector
    delta_axis = delta_axis/np.sqrt((delta_axis**2).sum(axis=1).reshape(-1,1))  # normalize the vector
    
    a_projection = (tangential * alpha_axis).sum(axis=1)  # get the projection factor for each epoch in RA
    d_projection = (tangential * delta_axis).sum(axis=1)  # in DEC 
    
    return a_projection, d_projection

In [ ]:
ra = ?
dec = ?
epoch = ?
dir_or_normal_vector = 'Gaia/'

a_projection, d_projection = get_scan_angle(ra, dec, epoch, dir_of_normal_vector)